In [ ]:
# Import dependencies
import torch
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler

import pcfg_models as pcfg_models
import pcfg_loader as loader
import source as source


scaler = GradScaler()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

In [ ]:
loader_params = {
    'datafolder': 'training',
    'batch_size': 1,
    'shuffle': True,
    'only_inputs': True,
    'print_steps': False,
    'moves_per_step': 1,
    'max_steps': 1,
    'p_use_base': 0.1,
}

data_yielder = loader.get_pcfg_datayielder(**loader_params)

In [ ]:
# Instantiate our Encoder classifier
dsl_list = data_yielder.label_ops
embedding_model_fp = 'mobilenet_v2'
em = source.embedding.load_mobilenet_v2()

pcfg_encoder = pcfg_models.PCFG_Encoder(
    n_embd = 64, 
    n_head = 4, 
    n_layer = 6, 
    ff_hd = 128, 
    dropout = 0, # This may cause issues if you make this non-zero
    block_size = 35, 
    dsl_mapping = dsl_list, 
    embedding_model_fp = embedding_model_fp,
    freeze_emb_model=True, 
    input_projection = True,
    input_dim_size,
    device = device
)
pcfg_encoder = pcfg_encoder.to(device)
em = None   # Clear the model from memory

# Use this to load in a pre-trained model instead
# filename = ?
# pcfg_encoder = pcfg_models.PCFG_Encoder.load_model(f"trained_pcfg_models/{filename}", print_statements=True, device=device)